In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.tests import *
import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("recommender").enableHiveSupport().getOrCreate()

In [3]:
sc = spark.sparkContext

# Preparing the Data

In [4]:
rawUserArtistData = sc.textFile("profiledata_06-May-2005/user_artist_data.txt", 20)

In [5]:
rawUserArtistData.getNumPartitions()

20

In [6]:
rawUserArtistData.first()

u'1000002 1 55'

In [7]:
def parse(line):
    line = line.split(' ')
    user = int(line[0])
    artist = int(line[1])
    return (user, artist)

In [8]:
userArtistDF = rawUserArtistData.map(parse).toDF(["user", "artist"])

In [9]:
userArtistDF.show()

+-------+-------+
|   user| artist|
+-------+-------+
|1000002|      1|
|1000002|1000006|
|1000002|1000007|
|1000002|1000009|
|1000002|1000010|
|1000002|1000013|
|1000002|1000014|
|1000002|1000017|
|1000002|1000024|
|1000002|1000025|
|1000002|1000028|
|1000002|1000031|
|1000002|1000033|
|1000002|1000042|
|1000002|1000045|
|1000002|1000054|
|1000002|1000055|
|1000002|1000056|
|1000002|1000059|
|1000002|1000062|
+-------+-------+
only showing top 20 rows



In [10]:
userArtistDF.cache()

DataFrame[user: bigint, artist: bigint]

In [11]:
userArtistDF.agg(
        min("user"), max("user"), min("artist"), max("artist")).show()

+---------+---------+-----------+-----------+
|min(user)|max(user)|min(artist)|max(artist)|
+---------+---------+-----------+-----------+
|       90|  2443548|          1|   10794401|
+---------+---------+-----------+-----------+



In [12]:
userArtistDF.describe().show()

+-------+------------------+------------------+
|summary|              user|            artist|
+-------+------------------+------------------+
|  count|          24296858|          24296858|
|   mean|1947573.2653533637|1718704.0937568964|
| stddev|496000.55518191896|2539389.0924283494|
|    min|                90|                 1|
|    max|           2443548|          10794401|
+-------+------------------+------------------+



In [13]:
rawArtistData = spark.read.text("profiledata_06-May-2005/artist_data.txt")

In [14]:
rawArtistData = sc.textFile("profiledata_06-May-2005/artist_data.txt", 20)

In [15]:
rawArtistData.take(5)

[u'1134999\t06Crazy Life',
 u'6821360\tPang Nakarin',
 u'10113088\tTerfel, Bartoli- Mozart: Don',
 u'10151459\tThe Flaming Sidebur',
 u'6826647\tBodenstandig 3000']

In [28]:
def parse_artist(line):
    line = line.split('\t')
    ID = line[0]
    name = line[1]
    return (int(ID), name) #if ID and name else (-9999999, name)
    

In [29]:
rawArtistDataDF = rawArtistData.map(parse_artist).toDF(["ID", "name"])

In [30]:
rawArtistDataDF.show()

+--------+--------------------+
|      ID|                name|
+--------+--------------------+
| 1134999|        06Crazy Life|
| 6821360|        Pang Nakarin|
|10113088|Terfel, Bartoli- ...|
|10151459| The Flaming Sidebur|
| 6826647|   Bodenstandig 3000|
|10186265|Jota Quest e Ivet...|
| 6828986|       Toto_XX (1977|
|10236364|         U.S Bombs -|
| 1135000|artist formaly kn...|
|10299728|Kassierer - Musik...|
|10299744|         Rahzel, RZA|
| 6864258|      Jon Richardson|
| 6878791|Young Fresh Fello...|
|10299751|          Ki-ya-Kiss|
| 6909716|Underminded - The...|
|10435121|             Kox-Box|
| 6918061|  alexisonfire [wo!]|
| 1135001|         dj salinger|
| 6940391|The B52's - Chann...|
|10475396|             44 Hoes|
+--------+--------------------+
only showing top 20 rows



In [31]:
rawArtistAlias = sc.textFile("profiledata_06-May-2005/artist_alias.txt", 20)

In [32]:
def parse_artist_alias(line):
    line = line.split('\t')
    artist = line[0]
    alias = line[1]
    return (int(artist), int(alias)) if (artist and alias) else (-9999999, -9999999)

In [33]:
rawArtistAliasRDD = rawArtistAlias.map(parse_artist_alias)

In [34]:
rawArtistAliasRDD.first()

(1092764, 1000311)

In [35]:
rawArtistAliasDF = rawArtistAliasRDD.toDF(["artist", "alias"])

In [36]:
rawArtistAliasDF.show()

+--------+-------+
|  artist|  alias|
+--------+-------+
| 1092764|1000311|
| 1095122|1000557|
| 6708070|1007267|
|10088054|1042317|
| 1195917|1042317|
| 1112006|1000557|
| 1187350|1294511|
| 1116694|1327092|
| 6793225|1042317|
| 1079959|1000557|
| 6789612|1000591|
| 1262241|1000591|
| 6791455|1000591|
| 6694867|1000591|
|10141141|1113738|
| 1295140|1000591|
| 1027859|1252408|
| 2127019|1000591|
| 2153974|1000591|
| 1232342|1000591|
+--------+-------+
only showing top 20 rows



In [37]:
artistByID = rawArtistDataDF

In [38]:
artistAlias = rawArtistAliasDF

artistByID.filter(artistByID.ID.isin(1208690, 1003926)).show()

In [39]:
artistByID[artistByID.ID.isin([1208690, 1003926])].show()

Py4JJavaError: An error occurred while calling o258.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 19.0 failed 1 times, most recent failure: Lost task 0.0 in stage 19.0 (TID 57, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.1.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.1.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.1.1/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-28-d0e8da487a69>", line 4, in parse_artist
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:1

In [41]:
from pyspark.ml.recommendation import *

In [45]:
model = ALS(rank=10, maxIter=5, seed=0)

In [46]:
als = model.fit()

ALS_4a5ebae6c790f4765298